In [1]:
import numpy as np

In [2]:
country = []
with open('ElectData.txt') as f:
    data = f.read().splitlines()
    for line in data[1:]:
        state = line.split('\t')
        if line != 'D.C.\tD\tD\tD\tD\tD\tD\tD\tD\tD\tD\tD\tD':
            country.append(state[1:])

## Estimate a two-state discrete Markov chain.

In [3]:
def count (start, transfer, rangelist):
    num = 0
    for i in range(1,len(rangelist)-1):
        if rangelist[i-1]==start and rangelist[i]==transfer:
            num += 1
    return num

In [4]:
RR = RD = DR = DD = 0
for state in country:
    RR += count('R','R',state)
    RD += count('R','D',state)
    DR += count('D','R',state)
    DD += count('D','D',state)
tranMCountry = np.array([[RR, RD], [DR, DD]])
rowsum = tranMCountry.sum(axis=1)
proMCountry = np.dot(np.diag(1./rowsum),tranMCountry)

In [5]:
tranMCountry

array([[264,  65],
       [ 40, 131]])

In [6]:
proMCountry

array([[ 0.80243161,  0.19756839],
       [ 0.23391813,  0.76608187]])

## 2. Estimate the steady-state distribution for the chain.

In [7]:
# Finding steady-state probabilities
P2 = proMCountry.dot(proMCountry)
for i in range(9):
    print("Chance of being X after %d iterations:"%(i+2))
    print(P2)
    P2 = P2.dot(P2)

Chance of being X after 2 iterations:
[[ 0.69011132  0.30988868]
 [ 0.36690374  0.63309626]]
Chance of being X after 3 iterations:
[[ 0.58995295  0.41004705]
 [ 0.48548981  0.51451019]]
Chance of being X after 4 iterations:
[[ 0.54711814  0.45288186]
 [ 0.5362056   0.4637944 ]]
Chance of being X after 5 iterations:
[[ 0.54217605  0.45782395]
 [ 0.54205697  0.45794303]]
Chance of being X after 6 iterations:
[[ 0.54212153  0.45787847]
 [ 0.54212152  0.45787848]]
Chance of being X after 7 iterations:
[[ 0.54212152  0.45787848]
 [ 0.54212152  0.45787848]]
Chance of being X after 8 iterations:
[[ 0.54212152  0.45787848]
 [ 0.54212152  0.45787848]]
Chance of being X after 9 iterations:
[[ 0.54212152  0.45787848]
 [ 0.54212152  0.45787848]]
Chance of being X after 10 iterations:
[[ 0.54212152  0.45787848]
 [ 0.54212152  0.45787848]]


## What is the share of elections won by the Democrat?

In [8]:
ob2012 = [state[-1] for state in country]
with open('eleVote.txt') as f:
    data = f.read()
    string = data.split('\n')
    elector = [int(i) for i in string]

In [9]:
def prediction(init,proM,electorCount):
    preState = []
    simuState = np.random.rand(len(init))
    for i in range(len(init)):
        if init[i] == 'R':
            if simuState[i]>proM[0,0]:
                preState.append('D')
            else:
                preState.append('R')
        if init[i] == 'D':
            if simuState[i]>proM[1,1]:
                preState.append('R')
            else:
                preState.append('D')
    R = 0
    D = 0
    for i in range(len(init)):
        if preState[i] == 'R':
            R+=electorCount[i]
        else:
            D+=electorCount[i]
    if D>R:
        return 1
    if R>D:
        return 0
    if D==R:
        print('Need select agian if assumed there are only two parties.')
        return 0.5

In [10]:
Democrat = 0
for i in range(100):
    a = prediction(ob2012,proMCountry,elector)
    Democrat+=a
print('Democrat wins: %d, Republic wins: %d'%(Democrat,100-Democrat))

Democrat wins: 27, Republic wins: 73
